<a href="https://colab.research.google.com/github/monoramasn/Speech_fairness/blob/main/ro_whisper_medium_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install git+https://github.com/huggingface/transformers
!pip install librosa
!pip install evaluate>=0.30
!pip install jiwer
!pip install gradio

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-7ym6yknh
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-7ym6yknh
  Resolved https://github.com/huggingface/transformers to commit 307cf3a2ab3faf50fed5b3aabf180a4c98188587
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.36.0.dev0-py3-none-any.whl size=8049194 sha256=72aef7f587fc7135baf68bd6de2617c9e2f28587ebbe45421eb21a68794f693a
  Stored in directory: /tmp/pip-ephem-wheel-cache-8h144vmw/wheels/c0/14/d6/6c9a5582d2ac191ec0a483be151a4495fe1eb2a6706ca49f1b
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [6]:
import torch
from transformers import AutoFeatureExtractor, WhisperModel
from datasets import load_dataset

In [7]:
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import torchaudio

from tqdm.notebook import tqdm


In [8]:
voxpopuli_data = load_dataset("facebook/voxpopuli", "ro", split="test")
#column_name = "audio, normalized_text"

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [9]:
voxpopuli_data

Dataset({
    features: ['audio_id', 'language', 'audio', 'raw_text', 'normalized_text', 'gender', 'speaker_id', 'is_gold_transcript', 'accent'],
    num_rows: 1383
})

In [10]:
!pip install openai-whisper
import whisper
#model = whisper.load_model("base")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.6/798.6 kB 10.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 22.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=801356 sha256=ccee256a287487d5bf183fb1c3b69d7d5412cc80921c934e3b0bfc0c2569bc45
  Stored in directory: /root/.cache/pip/wheels/d0/85/e1/9361b4cbea7dd4b7f6702fa4c3afc94877952eeb2b62f45f56
Successfully built openai-whisper
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires openai, which is not installed.


In [11]:
model = whisper.load_model("medium")

100%|█████████████████████████████████████| 1.42G/1.42G [00:40<00:00, 38.0MiB/s]


In [12]:
from transformers import WhisperForConditionalGeneration, WhisperFeatureExtractor,WhisperTokenizer
from datasets import load_dataset
import torch
from tqdm import tqdm
from jiwer import wer

In [13]:
Gender=voxpopuli_data['gender']
Audio=voxpopuli_data['audio']
text=voxpopuli_data['normalized_text']
Gender=pd.DataFrame(Gender)
Audio=pd.DataFrame(Audio)
text=pd.DataFrame(text)

In [14]:
text

,0
0,vorbim pe drept cuvânt despre noua zeelandă ca...
1,raportul comitetului de supervizare e foarte n...
2,comisia a anunțat publicarea evaluării impactu...
3,este preferabil să investim în construcia de c...
4,această măsură ar urma și ar permite să realiz...
...,...
1378,aplicarea acestor lucruri care sunt deja în le...
1379,singurul argument prezentat de chișinău în fav...
1380,în ceea ce privește raportul privind protecția...
1381,s a încălcat astfel articolul douăzeci din con...


In [15]:
Gender.columns=['Gender']
text.columns=['Transcript']

In [16]:
file=pd.concat([Gender, text, Audio],axis=1)
file

,Gender,Transcript,path,array,sampling_rate
0,male,vorbim pe drept cuvânt despre noua zeelandă ca...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0079345703125, 0.000152587890625, -0.005004...",16000
1,female,raportul comitetului de supervizare e foarte n...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00067138671875, -0.001556396484375, -0.0028...",16000
2,female,comisia a anunțat publicarea evaluării impactu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007659912109375, -0.000885009765625, -0.009...",16000
3,female,este preferabil să investim în construcia de c...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00140380859375, -0.0010986328125, 0.0, -0....",16000
4,male,această măsură ar urma și ar permite să realiz...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002166748046875, 0.001708984375, 0.00146484...",16000
...,...,...,...,...,...
1378,female,aplicarea acestor lucruri care sunt deja în le...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0250244140625, -0.01055908203125, -0.029266...",16000
1379,male,singurul argument prezentat de chișinău în fav...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002960205078125, 0.002166748046875, 0.00234...",16000
1380,male,în ceea ce privește raportul privind protecția...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00115966796875, -0.004852294921875, -0.001...",16000
1381,male,s a încălcat astfel articolul douăzeci din con...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.005859375, -0.002197265625, -0.017059326171...",16000


In [17]:
male_samples=file.loc[file['Gender'] == 'male']
male_samples

,Gender,Transcript,path,array,sampling_rate
0,male,vorbim pe drept cuvânt despre noua zeelandă ca...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0079345703125, 0.000152587890625, -0.005004...",16000
4,male,această măsură ar urma și ar permite să realiz...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002166748046875, 0.001708984375, 0.00146484...",16000
5,male,doamnă președinte dacă vrem să realizăm cu ade...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.001251220703125, 0.000762939453125, 9.15527...",16000
8,male,cât despre conținutul modificărilor soluțiile ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00177001953125, 0.0, 0.000335693359375, 0....",16000
9,male,rezoluia pe care am votat o astăzi transmite u...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00201416015625, 0.0025634765625, 0.00210571...",16000
...,...,...,...,...,...
1377,male,nu în ultimul rând consider că parteneriatul s...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00616455078125, -0.006622314453125, -0.006...",16000
1379,male,singurul argument prezentat de chișinău în fav...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.002960205078125, 0.002166748046875, 0.00234...",16000
1380,male,în ceea ce privește raportul privind protecția...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00115966796875, -0.004852294921875, -0.001...",16000
1381,male,s a încălcat astfel articolul douăzeci din con...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.005859375, -0.002197265625, -0.017059326171...",16000


In [18]:
female_samples=file.loc[file['Gender'] == 'female']
female_samples

,Gender,Transcript,path,array,sampling_rate
1,female,raportul comitetului de supervizare e foarte n...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00067138671875, -0.001556396484375, -0.0028...",16000
2,female,comisia a anunțat publicarea evaluării impactu...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.007659912109375, -0.000885009765625, -0.009...",16000
3,female,este preferabil să investim în construcia de c...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.00140380859375, -0.0010986328125, 0.0, -0....",16000
6,female,propaganda rusă iarăși trebuie contracarată se...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.000396728515625, 0.00439453125, 0.00531005...",16000
7,female,ameliorarea formării agenilor de control un ni...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.00152587890625, 0.003265380859375, 0.001312...",16000
...,...,...,...,...,...
1370,female,apreciem propunerea comisie de a combina aciun...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.002105712890625, 0.00189208984375, 0.00631...",16000
1372,female,consecinele crizei actuale au un impact negati...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.001983642578125, -0.001312255859375, -0.00...",16000
1374,female,de aceea regulamentul prevede obligaia pentru ...,/root/.cache/huggingface/datasets/downloads/ex...,"[0.0006103515625, -0.00018310546875, 0.0020751...",16000
1376,female,și dumneavoastră domnule comisar avramopoulos ...,/root/.cache/huggingface/datasets/downloads/ex...,"[-0.070220947265625, -0.062896728515625, -0.07...",16000


In [19]:
def transcribe_audio(file_path):
    result = model.transcribe(file_path, language="ro")
    return result["text"]

In [20]:
def calculate_wer(ground_truths, hypotheses):
    return wer(ground_truths, hypotheses)

In [21]:
total_wer = 0
num_sample = 0

for index,row in male_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.25
Processed 2 files, WER: 0.23809523809523808
Processed 3 files, WER: 0.3870967741935484
Processed 4 files, WER: 0.2
Processed 5 files, WER: 0.3103448275862069
Processed 6 files, WER: 0.16
Processed 7 files, WER: 0.2
Processed 8 files, WER: 0.36363636363636365
Processed 9 files, WER: 0.16
Processed 10 files, WER: 0.4
Processed 11 files, WER: 0.29411764705882354
Processed 12 files, WER: 0.21428571428571427
Processed 13 files, WER: 0.6153846153846154
Processed 14 files, WER: 0.3
Processed 15 files, WER: 0.34285714285714286
Processed 16 files, WER: 0.391304347826087
Processed 17 files, WER: 0.3125
Processed 18 files, WER: 0.32608695652173914
Processed 19 files, WER: 0.5
Processed 20 files, WER: 0.23809523809523808
Processed 21 files, WER: 0.3888888888888889
Processed 22 files, WER: 0.25
Processed 23 files, WER: 0.16129032258064516
Processed 24 files, WER: 0.3142857142857143
Processed 25 files, WER: 0.5185185185185185
Processed 26 files, WER: 0.6333333333333333
P

In [22]:
total_wer = 0
num_sample = 0

for index,row in female_samples.iterrows():
    audio_path=row['path']
    ground_truth_text = row['Transcript']
    transcription = transcribe_audio(audio_path)
    # Compute metrics
    sample_wer = calculate_wer(ground_truth_text, transcription)

    total_wer += sample_wer
    num_sample += 1

    print(f"Processed {num_sample} files, WER: {sample_wer}")

# Calculate average metrics
average_wer = total_wer / num_sample

print(f"Average WER: {average_wer}")

Processed 1 files, WER: 0.25
Processed 2 files, WER: 0.26666666666666666
Processed 3 files, WER: 0.391304347826087
Processed 4 files, WER: 0.25
Processed 5 files, WER: 0.3125
Processed 6 files, WER: 0.5263157894736842
Processed 7 files, WER: 0.3103448275862069
Processed 8 files, WER: 0.2647058823529412
Processed 9 files, WER: 0.10869565217391304
Processed 10 files, WER: 0.28846153846153844
Processed 11 files, WER: 0.29411764705882354
Processed 12 files, WER: 0.2222222222222222
Processed 13 files, WER: 0.29310344827586204
Processed 14 files, WER: 0.4878048780487805
Processed 15 files, WER: 0.25
Processed 16 files, WER: 0.2
Processed 17 files, WER: 0.5384615384615384
Processed 18 files, WER: 0.0
Processed 19 files, WER: 0.16666666666666666
Processed 20 files, WER: 1.0
Processed 21 files, WER: 0.1111111111111111
Processed 22 files, WER: 0.14285714285714285
Processed 23 files, WER: 0.42857142857142855
Processed 24 files, WER: 0.23529411764705882
Processed 25 files, WER: 1.0
Processed 26 fi